In [1]:
import imageio
from PIL import Image
import numpy as np

# Function to compare two frames
def is_duplicate(frame1, frame2):
    return np.array_equal(frame1, frame2)

# Read the GIF
gif_path = 'concatenated_high_res.gif'
gif = imageio.get_reader(gif_path)

unique_frames = []
prev_frame = None

# Extract unique frames
for i, frame in enumerate(gif):
    if prev_frame is None or not is_duplicate(prev_frame, frame):
        unique_frames.append(frame)
        prev_frame = frame

# Save unique frames as images
for i, frame in enumerate(unique_frames):
    frame_image = Image.fromarray(frame)
    frame_image.save(f'unique_frame_{i}.png')

print(f"Extracted {len(unique_frames)} unique frames.")


Extracted 534 unique frames.


In [4]:
import imageio
from PIL import Image, ImageDraw
import os

# Define the path to the unique frames directory
unique_frames_path = 'unique_frames'

# Load unique frames
unique_frames = []
for filename in sorted(os.listdir(unique_frames_path)):
    if filename.endswith('.png'):
        frame = Image.open(os.path.join(unique_frames_path, filename))
        unique_frames.append(frame)

print(len(unique_frames))

13


In [20]:
import imageio
from PIL import Image, ImageDraw
import os

# Define the path to the unique frames directory
unique_frames_path = 'unique_frames'

# Define the duration for each frame in seconds
SECONDS_PER_FRAME = 2  # You can adjust this value as needed

# Load unique frames
unique_frames = []
for filename in sorted(os.listdir(unique_frames_path)):
    if filename.endswith('.png'):
        frame = Image.open(os.path.join(unique_frames_path, filename))
        unique_frames.append(frame)

# Define the dimensions for the sidebar and combined image
frame_width, frame_height = unique_frames[0].size
sidebar_width = int(frame_width * 0.2)  # Fixed sidebar width
sidebar_height = frame_height

# Create a blank sidebar
sidebar = Image.new('RGB', (sidebar_width, sidebar_height), (255, 255, 255))

# Combine frames with the sidebar
combined_frames = []
thumbnail_height = sidebar_height // len(unique_frames)  # Height for each thumbnail

for i, frame in enumerate(unique_frames):
    # Calculate the correct aspect ratio for the thumbnail
    aspect_ratio = frame_width / frame_height
    thumbnail_width = sidebar_width
    thumbnail_height = int(sidebar_width / aspect_ratio)
    
    if thumbnail_height * len(unique_frames) > sidebar_height:
        thumbnail_height = sidebar_height // len(unique_frames)
        thumbnail_width = int(thumbnail_height * aspect_ratio)

    # Create the thumbnail with correct aspect ratio
    thumbnail = frame.resize((thumbnail_width, thumbnail_height), Image.ANTIALIAS)
    
    # Create a new sidebar image with the correct width and height
    sidebar = Image.new('RGB', (sidebar_width, sidebar_height), (255, 255, 255))

    # Paste all previous thumbnails into the new sidebar
    y_offset = 0
    for j in range(i + 1):
        prev_thumbnail_height = int(sidebar_width / aspect_ratio)
        if prev_thumbnail_height * len(unique_frames) > sidebar_height:
            prev_thumbnail_height = sidebar_height // len(unique_frames)
            prev_thumbnail_width = int(prev_thumbnail_height * aspect_ratio)
        else:
            prev_thumbnail_width = sidebar_width

        previous_thumbnail = unique_frames[j].resize((prev_thumbnail_width, prev_thumbnail_height), Image.ANTIALIAS)
        sidebar.paste(previous_thumbnail, (0, y_offset))
        y_offset += prev_thumbnail_height

    # Combine the current frame with the updated sidebar
    combined_image = Image.new('RGB', (frame_width + sidebar_width, frame_height))
    combined_image.paste(frame, (0, 0))
    combined_image.paste(sidebar, (frame_width, 0))
    
    combined_frames.append(combined_image)

# Convert SECONDS_PER_FRAME to milliseconds
duration_in_ms = SECONDS_PER_FRAME * 1000

# Save the combined frames as a new GIF
combined_gif_path = 'combined_gif_correct_aspect_ratio_cropped.gif'
combined_frames[0].save(
    combined_gif_path,
    save_all=True,
    append_images=combined_frames[1:],
    duration=duration_in_ms,  # Duration of each frame in milliseconds
    loop=0
)


/var/folders/jr/pzgvyjp95jdfzl0rzmg25j280000gn/T/ipykernel_55910/3445015588.py:41: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  thumbnail = frame.resize((thumbnail_width, thumbnail_height), Image.ANTIALIAS)
/var/folders/jr/pzgvyjp95jdfzl0rzmg25j280000gn/T/ipykernel_55910/3445015588.py:56: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  previous_thumbnail = unique_frames[j].resize((prev_thumbnail_width, prev_thumbnail_height), Image.ANTIALIAS)


In [21]:
import imageio
from PIL import Image

# Define the duration for each frame in seconds
SECONDS_PER_FRAME = 2  # You can adjust this value as needed

# Convert SECONDS_PER_FRAME to milliseconds
duration_in_ms = SECONDS_PER_FRAME * 1000

# Path to the combined GIF
combined_gif_path = 'combined_gif_correct_aspect_ratio_cropped.gif'

# Load the combined GIF
gif = imageio.mimread(combined_gif_path)

# Determine the dimensions for cropping
frame_width, frame_height = gif[0].shape[1], gif[0].shape[0]
crop_width = int(frame_width * 0.9)  # Remove the right 10%

# Crop each frame
cropped_frames = []
for frame in gif:
    frame_image = Image.fromarray(frame)
    cropped_frame = frame_image.crop((0, 0, crop_width, frame_height))
    cropped_frames.append(cropped_frame)

# Save the cropped frames as a new GIF with the correct duration
cropped_gif_path = 'cropped_combined_gif.gif'
cropped_frames[0].save(
    cropped_gif_path,
    save_all=True,
    append_images=cropped_frames[1:],
    duration=duration_in_ms,  # Duration of each frame in milliseconds
    loop=0
)

print(f"Cropped GIF saved to {cropped_gif_path}")


Cropped GIF saved to cropped_combined_gif.gif
